In [1]:
'''
Script to filter out necessary subset
'''

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


#loading in the data
df = pd.read_csv('/Users/tlh4362/Downloads/SARSCoV2Biobank-ClinicalData_DATA_2022-10-07_0905.csv', encoding='latin-1')

cook_county = pd.read_csv('/Users/tlh4362/Desktop/Final_datasets/cookcountyzip.csv')

seq = pd.read_csv('/Users/tlh4362/Desktop/Final Sequences/seq_final_list(updated).csv')

risk = pd.read_csv('/Users/tlh4362/Downloads/SARSCoV2Biobank-Justriskscores_DATA_2022-06-22_1310.csv')
vax = pd.read_csv('/Users/tlh4362/Downloads/SARSCoV2Biobank-Vaccine_DATA_2022-10-06_1614.csv')
labs = pd.read_csv('/Users/tlh4362/Downloads/SARSCoV2Biobank-Justlabs_DATA_2022-08-12_1528.csv')

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,7,16,17,27,31,32,33,34,38,39,40,41,43,46,47,48,49,50,51,52,53,54,55,56,57,70,76,77,79,80,82,83,84,86,87,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4,7,10,13,16,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Co

In [2]:
#removing duplicate sequences
to_drop = []
for bid, count in zip(seq['Biobank.ID'].value_counts().index, seq['Biobank.ID'].value_counts().values):
    if count > 1:
        to_drop.append(seq[seq['Biobank.ID']==bid].index[1])
print(seq.shape)
seq = seq.drop(to_drop, axis = 0)
print(seq.shape)


(2129, 22)
(2120, 22)


In [4]:
#merging sequencing data to clinical data
df['record_id'] = df['study_id']
df = pd.merge(df, seq.loc[:,['record_id','nextclade.clade']], how = 'left')
df['clade'] = df['nextclade.clade']
df = df.drop('nextclade.clade', axis = 1)

In [5]:
#removal of samples that are unlinked to medical record numbers
count = 0
rid_missing_mrn = []
for m in range(len(df['mrn'])):
    try:
        int(df['mrn'].iloc[m])
    except:
        rid_missing_mrn.append(df['mrn'].iloc[m])
        count += 1
print(count)

df = df.drop(df[df['mrn'].isin(rid_missing_mrn)].index)
df['mrn'] = df['mrn'].astype(int)

120


In [12]:
#limiting our dataset to March 17th, 2020 to March 17th, 2022
df['sample_date'] = pd.to_datetime(df['sample_date'])

df = df[(df['sample_date'] >= pd.to_datetime('2020-03-17')) & (df['sample_date'] <= pd.to_datetime('2022-03-17'))]

In [15]:
#dropping all patients below the age of 18

df['dob'] = pd.to_datetime(df['dob'])

df['age'] = df['sample_date'] - df['dob']

df['age'] = [x.days/365 for x in df['age']]

df = df[df['age']>=18]

(4, 94)
(104, 94)


In [19]:
#if a patient has multiple samples, we take the earliest sample so that we end with one sample per patient

to_drop = []

for i in range(len(df['mrn'].value_counts())):
    mrn = df['mrn'].value_counts().index[i]
    count = df['mrn'].value_counts().iloc[i]
    if count > 1:
        duplicates = df[df['mrn']==mrn]
        first_sample_date = duplicates['sample_date'].min()
        if len([x for x in list(duplicates['clade'].unique()) if isinstance(x, float) == False]) > 0:
            if len([x for x in list(duplicates['clade'].unique()) if isinstance(x, float) == False]) > 1:
            #first sample date if patient has had two sequences
                first_sample_date = duplicates[duplicates['clade'].notnull()]['sample_date'].min()
                to_drop.extend(list(duplicates[(duplicates['sample_date']!=duplicates['sample_date'].min())].index))
            else:
                first_sample_date = duplicates[duplicates['clade'].notnull()]['sample_date'].min()
                to_drop.extend(list(duplicates[(duplicates['clade'].isnull()) | (duplicates['sample_date'] != first_sample_date)].index))
        elif duplicates[duplicates['sample_date']== first_sample_date].shape[0] == 1:
            to_drop.extend(list(duplicates[duplicates['sample_date']!= first_sample_date].index))
        else:
            idx = duplicates[duplicates['sample_date']== first_sample_date].index[0]
            other_first_date = list(duplicates[duplicates['sample_date']== first_sample_date].index[1:])
            not_first_date = list(duplicates[duplicates['sample_date']!= first_sample_date].index)
            not_first_date.extend(other_first_date)
            to_drop.extend(duplicates.loc[not_first_date,:].index)

df = df.drop(to_drop, axis = 0)

In [23]:
#only keep sequences that have zipcodes associated with Cook County, IL.
df['zipcode'] = df['zipcode'].astype(str)
cook_county['Zip Code'] = cook_county['Zip Code'].astype(str)
new_zip = []
for z in df['zipcode']:
    if len(z) == 5:
        new_zip.append(z)
    else:
        new_zip.append(z[0:5])
df['zipcode'] = new_zip
df = df[df['zipcode'].isin(cook_county['Zip Code'])]

In [25]:
#if a patient is missing hospital, ICU or death we assume they are not admitted and alive.
df['ca_hosp_admit'] = df['ca_hosp_admit'].fillna(0)
df['ca_icu'] = df['ca_icu'].fillna(0)
df['ca_death'] = df['ca_death'].fillna(0)

In [30]:
#if the patients have had an intubation we assume they were also in the ICU
#if the patient was in the ICU we assume they were also hospitalized


idx = df[(df['ca_intub']==1) & (df['ca_icu']==0)].index

df.at[idx, 'ca_icu']= 1
df.at[idx, 'ca_icu_admit_dt'] = df.loc[idx, 'ca_intub_dt']

for mrn in df[(df['ca_intub']==1) & (df['ca_icu'].isnull())]['mrn']:
    idx = df[df['mrn']==mrn].index
    df.at[idx, 'ca_icu'] = 1
    df.at[idx, 'ca_icu_admit_dt'] = df[df['mrn']==mrn]['ca_intub_dt'].iloc[0]

for mrn in df[(df['ca_icu']==1) & (df['ca_hosp_admit']==0)]['mrn']:
    idx = df[df['mrn']==mrn].index
    df.at[idx, 'ca_hosp_admit'] = 1
    df.at[idx, 'ca_hosp_admit_dt'] = df[df['mrn']==mrn]['ca_icu_admit_dt'].iloc[0]
    
    
for mrn in df[(df['ca_icu']==1) & (df['ca_hosp_admit'].isnull())]['mrn']:
    idx = df[df['mrn']==mrn].index
    df.at[idx, 'ca_hosp_admit'] = 1
    df.at[idx, 'ca_hosp_admit_dt'] = df[df['mrn']==mrn]['ca_icu_admit_dt'].iloc[0]
    
    

In [35]:
#if the patients have an outcome date we assume they had the associated outcome

idx = df[(df['ca_hosp_admit_dt'].notnull()) & (df['ca_hosp_admit']!=1)].index

df.at[idx, 'ca_hosp_admit'] = 1

idx = df[(df['ca_supp_oxy_st'].notnull()) & (df['ca_supp_oxy']!=1)].index

df.at[idx, 'ca_supp_oxy'] = 1

idx = df[(df['ca_ecmo_dt'].notnull()) & (df['ca_ecmo']!=1)].index

df.at[idx, 'ca_ecmo'] = 1

In [41]:
df = df.rename({'ï»¿record_id':'record_id'})

In [42]:
df['record_id'] = df.iloc[:,0]

In [45]:
#dates were determined by calculating the 7-day moving average and picking the date with the lowest count before the beginning of a surge.

wave1 = pd.to_datetime('2020-03-17')
wave2 = pd.to_datetime('2020-09-15')
wave3 = pd.to_datetime('2021-03-15')
wave4 = pd.to_datetime('2021-06-30')
wave5 = pd.to_datetime('2021-10-23')


wave1_idx = df[(df['sample_date'] >= wave1) & (df['sample_date'] < wave2)].index
wave2_idx = df[(df['sample_date'] >= wave2) & (df['sample_date'] < wave3)].index
wave3_idx = df[(df['sample_date'] >= wave3) & (df['sample_date'] < wave4)].index
wave4_idx = df[(df['sample_date'] >= wave4) & (df['sample_date'] < wave5)].index
wave5_idx = df[(df['sample_date'] >= wave5)].index

df.at[wave1_idx,'wave']='wave1'
df.at[wave1_idx,'comorbid_sum_median'] = df.loc[wave1_idx, 'ca_comorbid_sum'].median()

df.at[wave2_idx,'wave']='wave2'
df.at[wave2_idx,'comorbid_sum_median'] = df.loc[wave2_idx, 'ca_comorbid_sum'].median()

df.at[wave3_idx,'wave']='wave3'
df.at[wave3_idx,'comorbid_sum_median'] = df.loc[wave3_idx, 'ca_comorbid_sum'].median()

df.at[wave4_idx,'wave']='wave4'
df.at[wave4_idx,'comorbid_sum_median'] = df.loc[wave4_idx, 'ca_comorbid_sum'].median()

df.at[wave5_idx,'wave']='wave5'
df.at[wave5_idx,'comorbid_sum_median'] = df.loc[wave5_idx, 'ca_comorbid_sum'].median()


In [48]:
df = df.sort_values(by='sample_date')

In [52]:
#mergining in vaccination data, risk scores and laboratory measurements
vax = vax[vax['record_id'].isin(df['record_id'])]
risk = risk[risk['record_id'].isin(df['record_id'])]
labs = labs[labs['record_id'].isin(df['record_id'])]

In [56]:
vax_merge = pd.merge(df, vax, on = 'record_id')
for c in vax_merge.columns:
    if '_x' in c:
        vax_merge = vax_merge.drop(c, axis = 1)
    elif '_y' in c:
        new_col = c.split('_y')[0]

In [59]:
vax_merge = vax_merge.rename({c:new_col}, axis = 1)
vax_merge['ca_vax1_date'] = pd.to_datetime(vax_merge['ca_vax1_date'])
vax_merge['ca_vax2_date'] = pd.to_datetime(vax_merge['ca_vax2_date'])
vax_merge['ca_vax3_date'] = pd.to_datetime(vax_merge['ca_vax3_date'])
vax_merge['ca_vax4_date'] = pd.to_datetime(vax_merge['ca_vax4_date'])
vax_merge['ca_vax5_date'] = pd.to_datetime(vax_merge['ca_vax5_date'])

In [61]:
#determining vaccination status. one dose of J&J is fully vaccinated whereas two doses of other vaccines are considered fully vaccinated
#the dosage is only considered effective if it is at least 14 days before sampling


vax_date = ['ca_vax1_date', 'ca_vax2_date', 'ca_vax3_date', 'ca_vax4_date', 'ca_vax5_date']
manufacturer = ['ca_vaccine', 'ca_vaccine2', 'ca_vaccine3', 'ca_vaccine4', 'ca_vaccine5']
#1 = pfizer, 2 = moderna, 3 = j&j, 4, 5

vax_status = []
for idx in vax_merge.index:
    dates = vax_merge.loc[idx, vax_date]
    sample_date = vax_merge.loc[idx, 'sample_date']
    num_doses = vax_merge.loc[idx, vax_date].notnull().sum()
    
    if num_doses >= 3:
        last_vaccinated_date = dates[dates.notnull()].iloc[-1]
        days_diff = sample_date - last_vaccinated_date
        days_diff = days_diff.days
        if days_diff >= 14:
            vax_status.append('Boosted') #boosted
        else:
            second_to_last_vaccinated_date = dates[dates.notnull()].iloc[-2]
            
            days_diff = sample_date - second_to_last_vaccinated_date
            days_diff = days_diff.days
            if days_diff >= 14: #if the second dose is over 14 days out from sampling
                if vax_merge.loc[idx, 'ca_vaccine'] == 3: #if first dose is j&j then the individual is considered boosted
                    vax_status.append('Boosted')
                else:
                    vax_status.append('Fully Vaccinated')
            else:
                third_to_last_vaccinated_date = dates[dates.notnull()].iloc[-3]
                days_diff = sample_date - third_to_last_vaccinated_date
                days_diff = days_diff.days
                if days_diff >= 14:
                    if vax_merge.loc[idx, 'ca_vaccine'] == 3: #if first dose is j&j then the individual is considered boosted
                        vax_status.append('Fully Vaccinated')
                    else:   
                        vax_status.append('Partially Vaccinated')
                else:
                    vax_status.append('Unvaccinated')
   
    elif num_doses == 2:
        last_vaccinated_date = dates[dates.notnull()].iloc[-1]
        days_diff = sample_date - last_vaccinated_date
        days_diff = days_diff.days
        if days_diff >= 14:
            if vax_merge.loc[idx, 'ca_vaccine'] == 3: #if first dose is j&j then the individual is considered boosted
                vax_status.append('Boosted')
            else:
                vax_status.append('Fully Vaccinated') #boosted
        else:
            second_to_last_vaccinated_date = dates[dates.notnull()].iloc[-2]
            days_diff = sample_date - second_to_last_vaccinated_date
            days_diff = days_diff.days
            if days_diff >= 14:
                if vax_merge.loc[idx, 'ca_vaccine'] == 3:
                    vax_status.append('Fully Vaccinated')
                else:
                    vax_status.append('Partially Vaccinated')
            else:
                vax_status.append('Unvaccinated')
    
    elif num_doses == 1:
        last_vaccinated_date = dates[dates.notnull()].iloc[0]
        days_diff = sample_date - last_vaccinated_date
        days_diff = days_diff.days
        if days_diff >= 14:
            if vax_merge.loc[idx, 'ca_vaccine'] == 3:
                vax_status.append('Fully Vaccinated')
            else:
                vax_status.append('Partially Vaccinated')
        else:
            vax_status.append('Unvaccinated')
    else:
        vax_status.append('Unvaccinated')
    
    

In [63]:
vax_merge['Vaccination_status'] = vax_status

In [67]:
#calculating total number of total doses per patient
#calculating total number of doses (vax date) before infection(sample date) per patient 

doses_dict = {}
dose_before_inf = {}
for p in [('ca_vaccine', 'ca_vax1_date'),
         ('ca_vaccine2', 'ca_vax2_date'),
         ('ca_vaccine3', 'ca_vax3_date'),
         ('ca_vaccine4', 'ca_vax4_date'),
         ('ca_vaccine5', 'ca_vax5_date')]:
    vax_merge[p[1]] = pd.to_datetime(vax_merge[p[1]])
    vax_status = []
    doses_before_infection = []
    total_doses = []
    for i in vax_merge.index:
        if str(vax_merge.loc[i, p[0]]) == 'nan' and str(vax_merge.loc[i, p[1]]) == 'NaT':
            vax_status.append(0) #both missing
            total_doses.append(0)
            doses_before_infection.append(0)
            
        elif str(vax_merge.loc[i, p[0]]) == 'nan' and str(vax_merge.loc[i, p[1]]) != 'NaT':
            vax_status.append(1) #manufacturer missing
            if pd.to_datetime(vax_merge.loc[i,  p[1]]) < vax_merge.loc[i, 'sample_date']: #if the vaccination date is before the sample date
                doses_before_infection.append(1)
            else:
                doses_before_infection.append(0)
            total_doses.append(1)
            
        elif str(vax_merge.loc[i, p[0]]) != 'nan' and str(vax_merge.loc[i, p[1]]) == 'NaT':
            vax_status.append(2) #date missing
            total_doses.append(1)
            doses_before_infection.append(None)
            
        elif str(vax_merge.loc[i, p[0]]) != 'nan' and str(vax_merge.loc[i, p[1]]) != 'NaT':
            if pd.to_datetime(vax_merge.loc[i,  p[1]]) < vax_merge.loc[i, 'sample_date']: #if the vaccination date is before the sample date
                doses_before_infection.append(1)
            else:
                doses_before_infection.append(0)
                
            vax_status.append(3) #both included
            total_doses.append(1)
        else:
            doses_before_infection.append(0)
            vax_status.append(4) #unsure
            total_doses.append(0)
            
    doses_dict[p[1].split('_date')[0]] = total_doses
    dose_before_inf[p[1].split('_date')[0]] = doses_before_infection
    vax_merge[p[1].split('_date')[0] + '_status'] = vax_status

#adding up each dose for the total number of doses before infection    
sum_list_bi = []
for k,v in dose_before_inf.items():
    if '1' in k:
        sum_list_bi = v
        next
    else:
        sum_list_bi = [a + b for a,b in zip(v, sum_list_bi)]
vax_merge['doses_before_infection'] = sum_list_bi

sum_list_total = []
for k,v in doses_dict.items():
    if '1' in k:
        sum_list_total = v
        next
    else:
        sum_list_total = [a + b for a,b in zip(v, sum_list_total)]
vax_merge['doses'] = sum_list_total

print('done')

done


In [70]:
#for comorbidities, if missing data we assume they don't have the comorbidity

for c in [ 'ca_asthma', 'ca_copd', 'ca_cancer', 'ca_cvd', 'ca_cld', 'ca_dm', 'ca_hiv', 'ca_htn' , 'ca_immune', 
                        'ca_renal', 'ca_comorbid_sum']:
    df[c] = df[c].fillna(0)

In [75]:
#merged vaccination data with the final dataset
df = pd.merge(df, vax_merge.loc[:,['record_id','ca_vaccine', 'ca_vax1_date', 'ca_vax1_site', 'ca_vaccine2',
       'ca_vax2_date', 'ca_vax2_site', 'ca_vaccine3', 'ca_vax3_date',
       'ca_vax3_site', 'ca_vaccine4', 'ca_vax4_date', 'ca_vax4_site',
       'ca_vaccine5', 'ca_vax5_date', 'ca_vax5_site', 'doses_before_infection',
'doses']], how = 'left')

In [76]:
df = df.drop_duplicates()